The objective is to derive a Bresenham-style implementation for drawing sheared ellipses, i.e. an algorithm which plots the sheared ellipse in a rasterised display with integer-only manipulations, avoiding any explicit square-root computations and avoiding multiplications, where possible.

References for the closely related Bresenham-circle algorithm can be found [here](https://en.wikipedia.org/wiki/Midpoint_circle_algorithm).

For simplicity the sheared ellipse is assumed to be centred at $(0,0)$.  The control points are the point $(A,0)$, and the highest point on the ellipse is $(S,B)$.  See page 133 of [BBC Basic Reference Manual](https://www.riscosopen.org/zipfiles/platform/common/BASICRefManual.3.pdf?1613981974) for further motivation. 


## Equation for sheared ellipse

For ellipse centred at $(0,0)$, which has its highest point at $(S,B)$ (where $S$ is "shear" and $B$ is "height"), and passes through point $(A,0)$, the equation of the ellipse is given by:
\begin{align}
x=\frac{yS \pm A\sqrt{B^2-y^2}}{B}
\end{align}
for $-B \leq y \leq B$.

This rearranges into
\begin{align}
(xB-yS)^2&=A^2(B^2-y^2)\\
D(x,y)&=(xB-yS)^2-A^2(B^2-y^2)=0
\end{align}
The equation of the ellipse is $D(x,y)=0$.  Note that for every point $(x,y)$ inside the ellipse, $D(x,y)<0$, and for every point $(x,y)$ outside of the ellipse, $D(x,y)>0$.  

For each given $y$ coordinate, there are two values of $x$ which solve the ellipse equation $D(x,y)=0$, corresponding to the left and right sides of the ellipse.  Let these two solutions be $x_L$ and $x_R$, with $x_R>x_L$.

Define the centre line which runs from  $(-S,-B)$ to $(S,B)$ as having x-coordinate $x_C$ for any given $y$.  This line has equation $x_C=\frac{S}{B}y$, where $x_C$ stands for "$x$ centre". It needs to be that for any $y$, $x_L\leq x_c \leq x_R$.  Also, $x_c$ will always be the midpoint between $x_L$ and $x_R$.

## Solution of sheared-ellipse equation, without computing square root 

To solve the ellipse equation $D(x,y)=0$ for plotting an ellipse in rasterised displays, and avoiding the square-root computation, for a given $y$ and trial $x_R$ value for a point on the right of the ellipse, if $x_R$ is a good solution then we should have $D(x_R+0.5,y)>0$ and $D(x_R-0.5,y)<0$. An equivalent condition holds for $x_L$ (but with reversed inequality signs).

At $y=0$, we have $x_L=-A$ and $x_R=A$.  Then for a different value of $y$, such that $-B\leq y\leq B$, we can solve $x_R$ by iterating the following two steps:


1. If $D(x_R-0.5,y)>0$ then we are clearly too far outside the ellipse, and, provided $x_R>x_C$, then we can update $x_R \leftarrow x_R-1$.

2. If $D(x_R+0.5,y)<0$ then we are clearly too far inside the ellipse, or if $x_R<x_C$, then we should update $x_R \leftarrow x_R+1$.

For any fixed $y$, we should keep applying the above two steps until no more changes to $x_R$ occur (i.e. until the above two steps converge).  At this time $x_R$ will be the correct $x$-coordinate for the right-hand side of the ellipse at the given $y$-coordinate.  (In step 1 above, the $x_R>x_C$ check is also needed, otherwise it is possible for the update $x_R \leftarrow x_R-1$ to skip right over very narrow tall ellipses, thus passing into the left region outside of the ellipse with $D<0$, and therefore entering an infinite loop of repeatedly applying step 1.)

To solve for $x_L$ at the given $y$, we iterate the following two steps until convergence:
3. If $D(x_L+0.5,y)>0$ then we are clearly too far outside the ellipse, and, provided $x_L<x_C$, then we can update $x_L \leftarrow x_L+1$.

4. If $D(x_L-0.5,y)<0$ then we are clearly too far inside the ellipse, or if $x_L>x_C$, then  we should update $x_L \leftarrow x_L-1$

Hence we can solve the ellipse without using any explicit square-root computations.

Note that in the above, to avoid having to repeatedly calculate $x\pm 0.5$, it is preferable to redefine $D(x,y)=((x+0.5)B-yS)^2-A^2(B^2-y^2)$.  Also define $T=(x+0.5)B-yS$.  Then all occurrences of  $D(x+0.5,y)$ become $D(x,y)$, and all occurrences  of $D(x-0.5,y)$ become $D(x,y)+B^2-2BT$.  Hence, with further abbreviating of $D_L=D(x_L,y)$ etc, then steps 1-4 above change to the following:


1. While $D_R+B^2-2BT_R>0$ and $x_R>x_C$, update $x_R \leftarrow x_R-1$.
2. While $D_R<0$ or $x_R<x_C$, update $x_R \leftarrow x_R+1$.
3. While $D_L>0$ and $x_L<x_C$, update $x_L \leftarrow x_L+1$.
4. While $D_L+B^2-2BT_L<0$ or $x_L>x_C$, update $x_L \leftarrow x_L-1$



## Incremental updates to avoid multiplications

A general useful result is that when a quantity $X$ changes to $X+K$, the corresponding squared quantity $X^2$ will change to $X^2+2KX+K^2$.

Let $T=(x+0.5)B-yS$, and $F=2BT-B^2$. Then, $D=T^2-A^2(B^2-y^2)=(xB+0.5B-yS)^2-A^2(B^2-y^2)$.

If $x$ increases by $\pm 1$, then $\Delta T=\pm B$, $\Delta D=\Delta (T^2)=\pm 2BT+B^2$, $\Delta F=2B\Delta T=\pm 2B^2$

If $y$ increases by 1, then $\Delta D=\Delta (T^2)+A^2\Delta(y^2)=-2ST+S^2+A^2\Delta(y^2)$, $\Delta (y^2)=2y+1$, $\Delta T=-S$, $\Delta F=2B \Delta T=-2SB$


Since when increasing $y$ by 1, we have $\Delta D=-2ST+S^2+(1+2y)A^2$, we can define $dD=\Delta D$ as a variable of its own.  Therefore whenever $y$ increases by 1, we have:
$$\Delta (dD)=-2S\Delta T+2A^2=2(S^2+A^2)$$

And whenever $x$ increases by $\pm 1$, we have 
$$\Delta (dD)=-2S\Delta T=\mp 2SB$$

When y=0, we can initialise our variables with $x=A$, $T=0.5B+xB$, $D=T^2-A^2B^2$, $F=2BT-B^2$, $dD=-2ST+S^2+A^2$.  (These appear in lines 1 and 2 of the pseudocode below.)

To calculate $x_C$, we can use a Bresenham straight-line update, e.g. define $E_C=(x+0.5)B-yS$, set $x_C=0$, and then every time $y$ increases by 1, update $E_C$ and $x_C$ with lines 3,13 and 14 of the pseudocode below.

The pseudocode for the sheared-ellipse plotter, without any multiplications in the main loop, is as follows:


1. $X_L=-A$; $T_L=X_LB+B/2$; $D_L=B^2/4+X_LB^2$; $F_L=2BT_L-B^2$; $dD_L=-2ST_L+(S^2+A^2)$
2. $X_R=+A$; $T_R=X_RB+B/2$; $D_R=B^2/4+X_RB^2$; $F_R=2BT_R-B^2$; $dD_R=-2ST_R+(S^2+A^2)$
3. $E_C=B/2$; $X_C=0$
4. For (y=0;y<=B;y++) {
5. &nbsp;&nbsp;While ($D_R-F_R>0$ and $x_R>x_C$) {$x_R\mathrel{-}= 1$; $D_R\mathrel{+}=-F_R$; $F_R\mathrel{-}= 2B^2$; $dD_R\mathrel{+}= 2SB$}
6. &nbsp;&nbsp;While ($D_R<0$ or $x_R<x_C$) {$x_R\mathrel{+}= 1$; $D_R\mathrel{+}=+F_R+2B^2$; $F_R\mathrel{+}= 2B^2$; $dD_R\mathrel{-}= 2SB$}
7. &nbsp;&nbsp;While ($D_L>0$ and $x_L<x_C$) {$x_L\mathrel{+}= 1$; $D_L\mathrel{+}=+F_L+2B^2$; $F_L\mathrel{+}= 2B^2$; $dD_L\mathrel{-}= 2SB$}
8. &nbsp;&nbsp;While ($D_L-F_L<0$ or $x_L>x_C$) {$x_L\mathrel{-}= 1$; $D_L\mathrel{+}=-F_L$; $F_L\mathrel{-}= 2B^2$; $dD_L\mathrel{+}= 2SB$}
9. &nbsp;&nbsp;Plot points at $(X_L,y)$, $(X_R,y)$, $(-X_L,-y)$, $(-X_R,-y)$
10. &nbsp;&nbsp;$D_L\mathrel{+}=dD_L$;$D_R\mathrel{+}=dD_R$
11. &nbsp;&nbsp;$dD_L\mathrel{+}=2(S^2+A^2)$; $dD_R\mathrel{+}=2(S^2+A^2)$
12. &nbsp;&nbsp;$F_L\mathrel{-}=2SB$; $F_R\mathrel{-}=2SB$
13. &nbsp;&nbsp;$E_C\mathrel{+}=ABS(S)$
14. &nbsp;&nbsp;While ($E_C>B$) {$X_C\mathrel{+}=SGN(S)$;$E_C\mathrel{-}=B$ }
15. }


[Example implementation](https://bbcmic.ro/#%7B%22v%22%3A1%2C%22program%22%3A%2210MODE%204%5Cn20PROCellipse%28160%2C128%2C14%2C20%2C-10%29%5Cn30END%5Cn40%3A%5Cn50DEFPROCellipse%28CX%25%2CCY%25%2CA%25%2CB%25%2CS%25%29%5Cn60VDU%2029%2CCX%25*4%3BCY%25*4%3B%5Cn70BB%25%3DB%25*B%25%3ABBdouble%25%3DBB%25*2%5Cn80AA%25%3DA%25*A%25%5Cn90SB%25%3DS%25*B%25%3ASS%25%3DS%25*S%25%3ASBdouble%25%3DSB%25*2%3ASSpAAdouble%25%3D%28SS%25%2BAA%25%29*2%5Cn110XL%25%3D-A%25%3AT1%25%3DXL%25*B%25%2BB%25DIV2%3ADL%25%3DBB%25DIV4%2BXL%25*BB%25%3AFL%25%3DB%25*T1%25*2-BB%25%3AdDL%25%3D-2*S%25*T1%25%2B%28SS%25%2BAA%25%29%5Cn120XR%25%3D%20A%25%3AT2%25%3DXR%25*B%25%2BB%25DIV2%3ADR%25%3DBB%25DIV4%2BXR%25*BB%25%3AFR%25%3DB%25*T2%25*2-BB%25%3AdDR%25%3D-2*S%25*T2%25%2B%28SS%25%2BAA%25%29%5Cn130EC%25%3DB%25DIV2%3AXC%25%3D0%5Cn140FOR%20Y%25%3D0%20TO%20B%25%5Cn150OXR%25%3DXR%25%3AOXL%25%3DXL%25%5Cn160%3A%5Cn170%3A%5Cn180%3A%5Cn190T2%25%3DXR%25*B%25-Y%25*S%25%2BB%25DIV2%5Cn191IF%20FR%25%3C%3ET2%25*B%25*2-BB%25%3ASTOP%20%5Cn192IF%20DR%25%3C%3E%28T2%25*T2%25-AA%25*BB%25%2BY%25*Y%25*AA%25%29%3APRINTDR%25%2C%28T2%25*T2%25-AA%25*BB%25%2BY%25*Y%25*AA%25%29%3ASTOP%5Cn193IF%20dDR%25%3C%3E%28-2*S%25*T2%25%2BSS%25%2B%281%2B2*Y%25%29*AA%25%29%3ASTOP%5Cn220%3A%5Cn240IF%20DR%25-FR%25%20%3E%200%20AND%20XR%25%3EXC%25%20PROCdec_xr%3AGOTO%20240%5Cn250IF%20DR%25%20%3C%200%20OR%20XR%25%3CXC%25%20PROCinc_xr%3AGOTO%20250%5Cn260%3A%5Cn270DX%25%3DSGN%28XR%25-OXR%25%29%5Cn280MOVE%20%20%28OXR%25%2BDX%25%29*4%2C%20Y%25*4%3ADRAW%20XR%25*4%2C%20Y%25*4%5Cn290MOVE%20-%28OXR%25%2BDX%25%29*4%2C-Y%25*4%3ADRAW-XR%25*4%2C-Y%25*4%5Cn300%3A%5Cn310T1%25%3DXL%25*B%25-Y%25*S%25%2BB%25DIV2%5Cn311IF%20FL%25%3C%3ET1%25*B%25*2-BB%25%3ASTOP%5Cn312IF%20DL%25%3C%3ET1%25*T1%25-AA%25*BB%25%2BY%25*Y%25*AA%25%3ASTOP%5Cn314IF%20dDL%25%3C%3E-2*S%25*T1%25%2BSS%25%2B%281%2B2*Y%25%29*AA%25%3ASTOP%5Cn340%3A%5Cn360IF%20DL%25%20%3E%200%20AND%20XL%25%3CXC%25%20PROCinc_xl%3AGOTO%20360%5Cn370IF%20DL%25-FL%25%20%3C%200%20OR%20XL%25%3EXC%25%20PROCdec_xl%3AGOTO%20370%5Cn380%3A%5Cn390DX%25%3DSGN%28XL%25-OXL%25%29%5Cn400MOVE%20%20%28OXL%25%2BDX%25%29*4%2C%20Y%25*4%3ADRAW%20XL%25*4%2C%20Y%25*4%5Cn410MOVE%20-%28OXL%25%2BDX%25%29*4%2C-Y%25*4%3ADRAW-XL%25*4%2C-Y%25*4%5Cn420%5Cn425REMPRINTXC%25%2C%28XL%25%2BXR%25%29DIV2%5Cn430EC%25%3DEC%25%2BABS%28S%25%29%5Cn440IF%20EC%25%3EB%25%3AXC%25%3DXC%25%2BSGN%28S%25%29%3AEC%25%3DEC%25-B%25%3AGOTO440%5Cn450DR%25%3DDR%25%2BdDR%25%3ADL%25%3DDL%25%2BdDL%25%5Cn455dDR%25%3DdDR%25%2BSSpAAdouble%25%3AdDL%25%3DdDL%25%2BSSpAAdouble%25%5Cn470FR%25%3DFR%25-SBdouble%25%3AFL%25%3DFL%25-SBdouble%25%5Cn480NEXT%5Cn490%3A%5Cn500PLOT%2069%2C%20S%25*4%2C%20B%25*4%5Cn510PLOT%2069%2C-S%25*4%2C-B%25*4%5Cn520%3A%5Cn530ENDPROC%5Cn540%3A%5Cn550DEFPROCinc_xl%3AXL%25%3DXL%25%2B1%3ADL%25%3DDL%25%2BFL%25%2BBBdouble%25%3AFL%25%3DFL%25%2BBBdouble%25%3AdDL%25%3DdDL%25-SBdouble%25%3AENDPROC%5Cn560DEFPROCdec_xl%3AXL%25%3DXL%25-1%3ADL%25%3DDL%25-FL%25%3AFL%25%3DFL%25-BBdouble%25%3AdDL%25%3DdDL%25%2BSBdouble%25%3AENDPROC%5Cn570DEFPROCinc_xr%3AXR%25%3DXR%25%2B1%3ADR%25%3DDR%25%2BFR%25%2BBBdouble%25%3AFR%25%3DFR%25%2BBBdouble%25%3AdDR%25%3DdDR%25-SBdouble%25%20%3AENDPROC%5Cn580DEFPROCdec_xr%3AXR%25%3DXR%25-1%3ADR%25%3DDR%25-FR%25%3AFR%25%3DFR%25-BBdouble%25%3AdDR%25%20%3D%20dDR%25%2BSBdouble%25%3AENDPROC%22%7D)

## Limiting variables to 16-bit accuracy

For a fast implementation on an 8-bit processor (e.g. 6502), then for speed, it's helpful if the integers being manipulated require as few-bytes storage as possible.  We can divide all quantites by an appopriate power of 2 to reduce the storage required for each integer down to just 16 bits.  This will slightly lower the pixel-accuracy of the curve, but seems to be a trade-off worth making.  

See [Prototype](https://bbcmic.ro/#%7B%22v%22%3A1%2C%22program%22%3A%2210MODE%204%5Cn20PROCellipse%28160%2C128%2C34%2C30%2C-19%29%5Cn30END%5Cn40%3A%5Cn50DEFPROCellipse%28CX%25%2CCY%25%2CA%25%2CB%25%2CS%25%29%5Cn60VDU%2029%2CCX%25*4%3BCY%25*4%3B%5Cn70BB%25%3DB%25*B%25%3ABBdouble%25%3DBB%25*2%5Cn80AA%25%3DA%25*A%25%5Cn90SB%25%3DS%25*B%25%3ASS%25%3DS%25*S%25%3ASBdouble%25%3DSB%25*2%3ASSpAAdouble%25%3D%28SS%25%2BAA%25%29*2%5Cn110XL%25%3D-A%25%3AT1%25%3DXL%25*B%25%2BB%25DIV2%3ADL%25%3DBB%25DIV4%2BXL%25*BB%25%3AFL%25%3DB%25*T1%25*2-BB%25%3AdDL%25%3D-2*S%25*T1%25%2B%28SS%25%2BAA%25%29%5Cn120XR%25%3D%20A%25%3AT2%25%3DXR%25*B%25%2BB%25DIV2%3ADR%25%3DBB%25DIV4%2BXR%25*BB%25%3AFR%25%3DB%25*T2%25*2-BB%25%3AdDR%25%3D-2*S%25*T2%25%2B%28SS%25%2BAA%25%29%5Cn122ROR%25%3D2%5Cn125IF%20ABS%28FL%25%29%3E2%5E15%20OR%20ABS%28FR%25%29%3E2%5E15%20OR%20ABS%28dDL%25%29%3E2%5E15%20OR%20ABS%28dDR%25%29%3E2%5E15%3ABBdouble%25%3DBBdouble%25DIVROR%25%3ASBdouble%25%3DSBdouble%25DIVROR%25%3ASSpAAdouble%25%3DSSpAAdouble%25DIVROR%25%3ADL%25%3DDL%25DIVROR%25%3ADR%25%3DDR%25DIVROR%25%3AFL%25%3DFL%25DIVROR%25%3AFR%25%3DFR%25DIVROR%25%3AdDL%25%3DdDL%25DIVROR%25%3AdDR%25%3DdDR%25DIVROR%25%3AGOTO125%5Cn130EC%25%3DB%25DIV2%3AXC%25%3D0%5Cn140FOR%20Y%25%3D0%20TO%20B%25-1%5Cn150OXR%25%3DXR%25%3AOXL%25%3DXL%25%5Cn160%3A%5Cn220%3A%5Cn240IF%20DR%25-FR%25%20%3E%200%20AND%20XR%25%3EXC%25%20PROCdec_xr%3AGOTO%20240%5Cn250IF%20DR%25%20%3C%200%20OR%20XR%25%3CXC%25%20PROCinc_xr%3AGOTO%20250%5Cn260%3A%5Cn270DX%25%3DSGN%28XR%25-OXR%25%29%5Cn280MOVE%20%20%28OXR%25%2BDX%25%29*4%2C%20Y%25*4%3ADRAW%20XR%25*4%2C%20Y%25*4%5Cn290MOVE%20-%28OXR%25%2BDX%25%29*4%2C-Y%25*4%3ADRAW-XR%25*4%2C-Y%25*4%5Cn300%3A%5Cn360IF%20DL%25%20%3E%200%20AND%20XL%25%3CXC%25%20PROCinc_xl%3AGOTO%20360%5Cn370IF%20DL%25-FL%25%20%3C%200%20OR%20XL%25%3EXC%25%20PROCdec_xl%3AGOTO%20370%5Cn380%3A%5Cn390DX%25%3DSGN%28XL%25-OXL%25%29%5Cn400MOVE%20%20%28OXL%25%2BDX%25%29*4%2C%20Y%25*4%3ADRAW%20XL%25*4%2C%20Y%25*4%5Cn410MOVE%20-%28OXL%25%2BDX%25%29*4%2C-Y%25*4%3ADRAW-XL%25*4%2C-Y%25*4%5Cn420%5Cn425REMPRINTXC%25%2C%28XL%25%2BXR%25%29DIV2%5Cn430EC%25%3DEC%25%2BABS%28S%25%29%5Cn440IF%20EC%25%3EB%25%3AXC%25%3DXC%25%2BSGN%28S%25%29%3AEC%25%3DEC%25-B%25%3AGOTO440%5Cn450DR%25%3DDR%25%2BdDR%25%3ADL%25%3DDL%25%2BdDL%25%5Cn455dDR%25%3DdDR%25%2BSSpAAdouble%25%3AdDL%25%3DdDL%25%2BSSpAAdouble%25%5Cn470FR%25%3DFR%25-SBdouble%25%3AFL%25%3DFL%25-SBdouble%25%5Cn480NEXT%5Cn490%3A%5Cn500OXR%25%3DXR%25%3AOXL%25%3DXL%25%3ADX%25%3DSGN%28XR%25-XL%25%29%3AY%25%3DB%25%5Cn510MOVE%20%20%28OXL%25%2BDX%25%29*4%2C%20Y%25*4%3ADRAW%20%28OXR%25-DX%25%29*4%2C%20Y%25*4%5Cn515MOVE%20-%28OXL%25%2BDX%25%29*4%2C-Y%25*4%3ADRAW-%28OXR%25-DX%25%29*4%2C-Y%25*4%5Cn520%3A%5Cn530ENDPROC%5Cn540%3A%5Cn550DEFPROCinc_xl%3AXL%25%3DXL%25%2B1%3ADL%25%3DDL%25%2BFL%25%2BBBdouble%25%3AFL%25%3DFL%25%2BBBdouble%25%3AdDL%25%3DdDL%25-SBdouble%25%3AENDPROC%5Cn560DEFPROCdec_xl%3AXL%25%3DXL%25-1%3ADL%25%3DDL%25-FL%25%3AFL%25%3DFL%25-BBdouble%25%3AdDL%25%3DdDL%25%2BSBdouble%25%3AENDPROC%5Cn570DEFPROCinc_xr%3AXR%25%3DXR%25%2B1%3ADR%25%3DDR%25%2BFR%25%2BBBdouble%25%3AFR%25%3DFR%25%2BBBdouble%25%3AdDR%25%3DdDR%25-SBdouble%25%20%3AENDPROC%5Cn580DEFPROCdec_xr%3AXR%25%3DXR%25-1%3ADR%25%3DDR%25-FR%25%3AFR%25%3DFR%25-BBdouble%25%3AdDR%25%20%3D%20dDR%25%2BSBdouble%25%3AENDPROC%22%7D), which includes 2 extra lines of code (lines 122 and 125)

TODO: Are there any counter-examples which cause this 16-bit approximation to fail???

This example also includes a hack to make the left and right of the ellipse join up perfectly, despite the lower number of bits used, and which also gains further speed, is to adjust the main loop in y to only run from 0 to B-1, and then artificially draw the top and bottom horizontal lines to complete the ellipse.

## Caching drawing of line segments
To minimise the number of OS calls, we can group together the plotting of consecutive points that align into straight lines, and plot those aligned points with single draw statements.  This allows us to draw the ellipse more efficiently.

See [example](https://bbcmic.ro/#%7B%22v%22%3A1%2C%22program%22%3A%2210MODE%201%5Cn20PROCellipse%28160%2C128%2C140%2C120%2C-55%29%5Cn30END%5Cn40%3A%5Cn50DEFPROCellipse%28CX%25%2CCY%25%2CA%25%2CB%25%2CS%25%29%5Cn60VDU%2029%2CCX%25*4%3BCY%25*4%3B%5Cn70BB%25%3DB%25*B%25%3ABBdouble%25%3DBB%25*2%5Cn80AA%25%3DA%25*A%25%5Cn90SB%25%3DS%25*B%25%3ASS%25%3DS%25*S%25%3ASBdouble%25%3DSB%25*2%3ASSpAAdouble%25%3D%28SS%25%2BAA%25%29*2%5Cn110XL%25%3D-A%25%3AT1%25%3DXL%25*B%25%2BB%25DIV2%3ADL%25%3DBB%25DIV4%2BXL%25*BB%25%3AFL%25%3DB%25*T1%25*2-BB%25%3AdDL%25%3D-2*S%25*T1%25%2B%28SS%25%2BAA%25%29%5Cn120XR%25%3D%20A%25%3AT2%25%3DXR%25*B%25%2BB%25DIV2%3ADR%25%3DBB%25DIV4%2BXR%25*BB%25%3AFR%25%3DB%25*T2%25*2-BB%25%3AdDR%25%3D-2*S%25*T2%25%2B%28SS%25%2BAA%25%29%5Cn122ROR%25%3D2%5Cn125IF%20ABS%28FL%25%29%3E2%5E15%20OR%20ABS%28FR%25%29%3E2%5E15%20OR%20ABS%28dDL%25%29%3E2%5E15%20OR%20ABS%28dDR%25%29%3E2%5E15%3ABBdouble%25%3DBBdouble%25DIVROR%25%3ASBdouble%25%3DSBdouble%25DIVROR%25%3ASSpAAdouble%25%3DSSpAAdouble%25DIVROR%25%3ADL%25%3DDL%25DIVROR%25%3ADR%25%3DDR%25DIVROR%25%3AFL%25%3DFL%25DIVROR%25%3AFR%25%3DFR%25DIVROR%25%3AdDL%25%3DdDL%25DIVROR%25%3AdDR%25%3DdDR%25DIVROR%25%3AGOTO125%5Cn130EC%25%3DB%25DIV2%3AXC%25%3D0%5Cn135OYL%25%3D0%3AOYR%25%3D0%3AOOXL%25%3DXL%25%3AOOXR%25%3DXR%25%3AODELTAXL%25%3D0%3AODELTAXR%25%3D0%3Acol1%25%3D1%3Acol2%25%3D2%3AREM%20used%20for%20straight%20line%20segments%5Cn140FOR%20Y%25%3D0%20TO%20B%25-1%5Cn150OXR%25%3DXR%25%3AOXL%25%3DXL%25%5Cn160%3A%5Cn170%3A%5Cn180%3A%5Cn220%3A%5Cn240IF%20DR%25-FR%25%20%3E%200%20AND%20XR%25%3EXC%25%20PROCdec_xr%3AGOTO%20240%5Cn250IF%20DR%25%20%3C%200%20OR%20XR%25%3CXC%25%20PROCinc_xr%3AGOTO%20250%5Cn260%3A%5Cn261DELTAXR%25%3DXR%25-OXR%25%5Cn270IF%20DELTAXR%25%3DODELTAXR%25%20AND%20Y%25%3CB%25-1%20AND%20ABS%28DELTAXR%25%29%3C20%3AGOTO310%3AREM%20we%20are%20building%20a%20straight-line%20segment%20so%20don't%20draw%20anything%20for%20now%5Cn275IF%20ABS%28ODELTAXR%25%29%3D1%20AND%20DELTAXR%25%3D0%20AND%20Y%25%3CB%25-1%20AND%20Y%25%3DOYR%25%2B1%20%3AODELTAXR%25%3DDELTAXR%25%3AOOXR%25%3DOXR%25%3AGOTO310%3AREM%20fix%20for%20consecutive%20vlines%5Cn280DX%25%3DSGN%28OXR%25-OOXR%25%29%5Cn290GCOL3%2Ccol2%25%3Acol2%25%3D3-col2%25%3AMOVE%20%28OOXR%25%2BDX%25%29*4%2COYR%25*4%3ADRAW%20OXR%25*4%2C%28Y%25-1%29*4%3AMOVE%20-%28OOXR%25%2BDX%25%29*4%2C-OYR%25*4%3ADRAW-OXR%25*4%2C-%28Y%25-1%29*4%3AOYR%25%3DY%25%3AOOXR%25%3DOXR%25%3AODELTAXR%25%3DDELTAXR%25%3AREM%20finish%20off%20old%20straight-line%20segment%5Cn300%3A%5Cn310%3A%5Cn340%3A%5Cn360IF%20DL%25%20%3E%200%20AND%20XL%25%3CXC%25%20PROCinc_xl%3AGOTO%20360%5Cn370IF%20DL%25-FL%25%20%3C%200%20OR%20XL%25%3EXC%25%20PROCdec_xl%3AGOTO%20370%5Cn380%3A%5Cn390DELTAXL%25%3DXL%25-OXL%25%5Cn400IF%20DELTAXL%25%3DODELTAXL%25%20AND%20Y%25%3CB%25-1%20AND%20ABS%28DELTAXL%25%29%3C20%3AGOTO430%3AREM%20we%20are%20building%20a%20straight-line%20segment%20so%20don't%20draw%20anything%20for%20now%5Cn401IF%20ABS%28ODELTAXL%25%29%3D1%20AND%20DELTAXL%25%3D0%20AND%20Y%25%3CB%25-1%20AND%20Y%25%3DOYL%25%2B1%20%3AODELTAXL%25%3DDELTAXL%25%3AOOXL%25%3DOXL%25%3AGOTO450%3AREM%20fix%20for%20consecutive%20vlines%5Cn410DX%25%3DSGN%28OXL%25-OOXL%25%29%5Cn415GCOL3%2Ccol1%25%3Acol1%25%3D3-col1%25%3AMOVE%20%28OOXL%25%2BDX%25%29*4%2COYL%25*4%3ADRAW%20OXL%25*4%2C%28Y%25-1%29*4%3AMOVE%20-%28OOXL%25%2BDX%25%29*4%2C-OYL%25*4%3ADRAW-OXL%25*4%2C-%28Y%25-1%29*4%3AOYL%25%3DY%25%3AOOXL%25%3DOXL%25%3AODELTAXL%25%3DDELTAXL%25%3AREM%20finish%20off%20old%20straight-line%20segment%5Cn420%3A%5Cn425REMPRINTXC%25%2C%28XL%25%2BXR%25%29DIV2%5Cn430EC%25%3DEC%25%2BABS%28S%25%29%5Cn440IF%20EC%25%3EB%25%3AXC%25%3DXC%25%2BSGN%28S%25%29%3AEC%25%3DEC%25-B%25%3AGOTO440%5Cn450DR%25%3DDR%25%2BdDR%25%3ADL%25%3DDL%25%2BdDL%25%5Cn455dDR%25%3DdDR%25%2BSSpAAdouble%25%3AdDL%25%3DdDL%25%2BSSpAAdouble%25%5Cn470FR%25%3DFR%25-SBdouble%25%3AFL%25%3DFL%25-SBdouble%25%5Cn480NEXT%5Cn481Y%25%3DB%25%5Cn482REM%20finish%20off%20old%20line%20segments%3A%5Cn485OOXR%25%3DOXR%25%3AOXR%25%3DXR%25%3ADX%25%3DSGN%28OXR%25-OOXR%25%29%5Cn486GCOL3%2Ccol2%25%3Acol2%25%3D3-col2%25%3AMOVE%20%28OOXR%25%2BDX%25%29*4%2COYR%25*4%3ADRAW%20OXR%25*4%2C%28Y%25-1%29*4%3AMOVE%20-%28OOXR%25%2BDX%25%29*4%2C-OYR%25*4%3ADRAW-OXR%25*4%2C-%28Y%25-1%29*4%3AOYR%25%3DY%25%3AOOXR%25%3DOXR%25%3AODELTAXR%25%3DDELTAXR%25%3AREM%20finish%20off%20old%20straight-line%20segment%5Cn492OOXL%25%3DOXL%25%3AOXL%25%3DXL%25%3ADX%25%3DSGN%28OXL%25-OOXL%25%29%5Cn493GCOL3%2Ccol1%25%3Acol1%25%3D3-col1%25%3AMOVE%20%28OOXL%25%2BDX%25%29*4%2COYL%25*4%3ADRAW%20OXL%25*4%2C%28Y%25-1%29*4%3AMOVE%20-%28OOXL%25%2BDX%25%29*4%2C-OYL%25*4%3ADRAW-OXL%25*4%2C-%28Y%25-1%29*4%3AOYL%25%3DY%25%3AOOXL%25%3DOXL%25%3AODELTAXL%25%3DDELTAXL%25%3AREM%20finish%20off%20old%20straight-line%20segment%5Cn495%5Cn500%3A%5Cn520REM%20finish%20off%20cap%20and%20tail%20hlines%20of%20ellipse%5Cn525GCOL3%2C3%5Cn530OXR%25%3DXR%25%3AOXL%25%3DXL%25%3ADX%25%3DSGN%28XR%25-XL%25%29%3AY%25%3DB%25%5Cn540MOVE%20%20%28OXL%25%2BDX%25%29*4%2C%20Y%25*4%3ADRAW%20%28OXR%25-DX%25%29*4%2C%20Y%25*4%5Cn550MOVE%20-%28OXL%25%2BDX%25%29*4%2C-Y%25*4%3ADRAW-%28OXR%25-DX%25%29*4%2C-Y%25*4%5Cn560%3A%5Cn570ENDPROC%5Cn580%3A%5Cn590DEFPROCinc_xl%3AXL%25%3DXL%25%2B1%3ADL%25%3DDL%25%2BFL%25%2BBBdouble%25%3AFL%25%3DFL%25%2BBBdouble%25%3AdDL%25%3DdDL%25-SBdouble%25%3AENDPROC%5Cn600DEFPROCdec_xl%3AXL%25%3DXL%25-1%3ADL%25%3DDL%25-FL%25%3AFL%25%3DFL%25-BBdouble%25%3AdDL%25%3DdDL%25%2BSBdouble%25%3AENDPROC%5Cn610DEFPROCinc_xr%3AXR%25%3DXR%25%2B1%3ADR%25%3DDR%25%2BFR%25%2BBBdouble%25%3AFR%25%3DFR%25%2BBBdouble%25%3AdDR%25%3DdDR%25-SBdouble%25%20%3AENDPROC%5Cn620DEFPROCdec_xr%3AXR%25%3DXR%25-1%3ADR%25%3DDR%25-FR%25%3AFR%25%3DFR%25-BBdouble%25%3AdDR%25%20%3D%20dDR%25%2BSBdouble%25%3AENDPROC%22%7D), which uses alternating colours to highlight the draw statements being made.

Comments/bugfixes? Please let us know.
Also, are there any references to similar ellipse algorithms?